In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
 !pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-ipcukgb8
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-ipcukgb8
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 801584cceb559adc54e828ebe9b385c5f53fe70f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-mjsqtjep/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [ ]:
# Install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

# Download VnCoreNLP pre-trained model
!wget -O VnCoreNLP-1.1.1.jar https://github.com/vncorenlp/VnCoreNLP/blob/master/VnCoreNLP-1.1.1.jar

--2024-07-05 02:17:39--  https://github.com/vncorenlp/VnCoreNLP/blob/master/VnCoreNLP-1.1.1.jar
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar     [ <=>                ] 276.38K  --.-KB/s    in 0.009s  

2024-07-05 02:17:39 (30.9 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [283010]



In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%capture
!pip install datasets
!pip install evaluate
!pip install transformers

In [ ]:
%%capture
# Tải code và cài thư viện transformer

# Tải code
%cd /content
!git clone https://github.com/huggingface/transformers

# Cài thư viện transformer
%cd transformers
!pip install .

In [ ]:
!pip install transformers torch
!pip install vncorenlp
!pip install pandas numpy scikit-learn torch transformers pyspark kafka-python
!pip install findspark
!pip install sodapy

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import datetime
from pyspark.sql import functions as f
from pyspark.conf import SparkConf
from sodapy import Socrata
from pyspark.sql.streaming import DataStreamReader
scala_version = '2.12'
spark_version = '3.5.0'
packages = [
    f'org.apache.spark:spark-sql-kafka-0-10_{scala_version}:{spark_version}',
    'org.apache.kafka:kafka-clients:3.6.1'
]
spark = SparkSession \
.builder \
.appName("Chronos") \
.master("local") \
.config("spark.executor.memory", "16g") \
.config("spark.driver.memory", "16g") \
.config("spark.python.worker.reuse", "true") \
.config("spark.sql.execution.arrow.pyspark.enabled", "true") \
.config("spark.sql.execution.arrow.maxRecordsPerBatch", "16") \
.config("spark.jars.packages", ",".join(packages)) \
.getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

conf=SparkConf()

spark

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, col, split
from pyspark.sql.types import StructType, StructField, StringType
from torch.utils.data import DataLoader
from transformers import TrainingArguments, Trainer
from pyspark.sql.functions import pandas_udf, PandasUDFType
from sklearn.model_selection import train_test_split
import pyspark
from pyspark.ml.feature import Tokenizer, CountVectorizer, StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType, ArrayType

**Khởi tạo Spark session**

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BERT Text Classification with Spark") \
    .getOrCreate()


In [ ]:
# Đọc dũ liệu CSV
data = pd.read_csv('/content/output.csv')
data

,Comment,Label
0,garnacho bảnh quá,1.0
1,bình an đăng lại kìa,0.0
2,gọi garnacho là gì,0.0
3,"chỉnh sửa đẹp , ngầu âm thanh chất lượng nhưng...",0.0
4,hâm mộ manchester city những vẫn thích united,1.0
...,...,...
5109,romelu lukaku khiến tôi bán xe rồi,2.0
5110,bỉ là đội yếu chứ không mạnh như mầy tưởng,2.0
5111,đồng tiền lái đôi chân đá trượt bóng luôn khi ...,2.0
5112,vứt lịch thi đấu mới quan trọng còn nhận định ...,2.0


In [ ]:
data = data[['Comment', 'Label']]
data = data[data['Label'].isin([0, 1, 2])].reset_index(drop=True)

**Chia dữ liệu thành tập huấn luyện và kiểm tra**

In [ ]:
#Chia dữ liệu thành tập huấn luyện và kiểm tra
train_df, test_df = train_test_split(data, test_size=0.2, random_state=42)

**Chuyển đổi DataFrame từ Pandas sang Spark**

In [ ]:
#Chuyển đổi DataFrame từ Pandas sang Spark
spark_train_df = spark.createDataFrame(train_df)
spark_test_df = spark.createDataFrame(test_df)


**Chuẩn bị dữ liệu và mô hình BERT cho nhiệm vụ phân loại**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import Dataset, DataLoader
import torch

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=3)

class CommentDataset(Dataset):
    def __init__(self, comments, labels, tokenizer, max_len):
        self.comments = comments
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.comments)

    def __getitem__(self, item):
        comment = str(self.comments[item])
        label = self.labels[item]

        encoding = self.tokenizer.encode_plus(
            comment,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt',
        )

        return {
            'comment_text': comment,
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def create_data_loader(df, tokenizer, max_len, batch_size):
    ds = CommentDataset(
        comments=df.Comment.to_numpy(),
        labels=df.Label.to_numpy(),
        tokenizer=tokenizer,
        max_len=max_len
    )

    return DataLoader(
        ds,
        batch_size=batch_size,
        num_workers=4
    )

MAX_LEN = 85
BATCH_SIZE = 16

train_data_loader = create_data_loader(train_df, tokenizer, MAX_LEN, BATCH_SIZE)
test_data_loader = create_data_loader(test_df, tokenizer, MAX_LEN, BATCH_SIZE)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


**Huấn luyện và đánh giá mô hình**

In [ ]:
from transformers import AdamW, get_linear_schedule_with_warmup
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5, correct_bias=False)
EPOCHS = 3
total_steps = len(train_data_loader) * EPOCHS

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

for epoch in range(EPOCHS):
    model.train()
    for batch in train_data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels
        )

        loss = outputs.loss
        loss.backward()

        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()

model.eval()
predictions = []
true_labels = []


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


In [ ]:
with torch.no_grad():
    for batch in test_data_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask
        )

        _, preds = torch.max(outputs.logits, dim=1)
        predictions.extend(preds)
        true_labels.extend(labels)

accuracy = accuracy_score(true_labels, predictions)
f1 = f1_score(true_labels, predictions, average='weighted')

print(f'Accuracy: {accuracy}')
print(f'F1 Score: {f1}')


Accuracy: 0.6373390557939914
F1 Score: 0.5894774453366388


In [ ]:
test_df['Predicted_Label'] = [pred.item() for pred in predictions]
test_df

,Comment,Label,Predicted_Label
1736,tối qua bồ đào nha nhiều ông đá như chơi không...,2.0,1
755,cristiano ronaldo là màu mè nhất,2.0,1
1669,quang hải đi qua nước ngoài nhặt bóng rồi,2.0,0
1322,yahh,0.0,1
1870,quá là xuất sắc việt namm ơii,1.0,0
...,...,...,...
1360,máy chạy top,1.0,1
1989,và con tim đã vui trở lại tuyệt vời việt nam ơi,1.0,1
786,di dimaria,1.0,1
2280,bỉ 2 1 pháp,0.0,0


# **Save CSV**

In [ ]:
# Lưu file csv
output_path = '/content/predicted_output_BERT.csv'
test_df.to_csv(output_path, index=False)


In [ ]:
import joblib

In [ ]:
# Lưu model và tokenizer vào tệp .pkl
model_path = '/content/phobert_model.pkl'
tokenizer_path = '/content/phobert_tokenizer.pkl'
joblib.dump(model, model_path)
joblib.dump(tokenizer, tokenizer_path)